In [50]:
# --- Step 1: Import Professional Libraries ---

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, Markdown

# Configure style
sns.set_theme(style="whitegrid", font="Calibri", font_scale=1.2)
plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12


In [51]:
# --- Step 2: Load the Cleaned Data Sheet ---

file_path = "DBTA_GP2_Survey_JSO.xlsx"
data = pd.read_excel(file_path, sheet_name="Cleaned Data")

# Display shape and preview
display(Markdown(f"### Data Shape: {data.shape[0]} rows × {data.shape[1]} columns"))
display(data.head())

### Data Shape: 47 rows × 169 columns

,No,Name of respondent (optional),Country,TVET Centre (if applicable),Job Title/Designation,Are you a professed Salesian?,Do you have any vocational or technical training?,In what field?,How many years of service have you worked as a Job Services Officer?,Do you have access to the JSO Quality Manual?,...,"List any other stakeholder forums, national round tables, working groups, and dialogues you have participated in over the last 3 years",In what ways do Don Bosco TVET centres collaborate with other TVET centres? (Choose all that apply) \n[Sharing best practices and teaching methodologies],In what ways do Don Bosco TVET centres collaborate with other TVET centres? (Choose all that apply) \n[Participating in joint training programmes for staff and trainees],"In what ways do Don Bosco TVET centres collaborate with other TVET centres? (Choose all that apply) \n[Accessing shared resources and materials (curriculum, equipment, digital platforms)]",In what ways do Don Bosco TVET centres collaborate with other TVET centres? (Choose all that apply) \n[Engaging in peer-to-peer learning and mentorship],In what ways do Don Bosco TVET centres collaborate with other TVET centres? (Choose all that apply) \n[Contributing to joint advocacy and policy initiatives],In what ways do Don Bosco TVET centres collaborate with other TVET centres? (Choose all that apply) \n[Taking part in regional or national TVET forums],In what ways do Don Bosco TVET centres collaborate with other TVET centres? (Choose all that apply) \n[They do not collaborate],In what ways do Don Bosco TVET centres collaborate with other TVET centres? (Choose all that apply) \n[Other (Please specify)],In what ways do Don Bosco TVET centres collaborate with other TVET centres? (Choose all that apply)
0,1,LUKA JOHN SIMON,South Sudan,DON BOSCO VACTATIONAL TRAINING CENTRE WAU,Local Job Services Officer,No,Yes,SOLAR INSTALLATION,4-6,Yes,...,GENDER OFFICE \nLABOUR OFFICE\nCOMMUNITY LEADE...,Sharing best practices and teaching methodologies,Participating in joint training programmes for...,Accessing shared resources and materials (curr...,Engaging in peer-to-peer learning and mentorship,Contributing to joint advocacy and policy init...,Taking part in regional or national TVET forums,NaN,NaN,Sharing best practices and teaching methodolog...
1,2,BUKASA Gabriel,Democratic Republic of Congo (DRC),ITIP-MOLK,Local Job Services Officer,Yes,Yes,Mécanique Générale,10 or more,Yes,...,"Les experts du ministère de l'éducation, du tr...",Sharing best practices and teaching methodologies,Participating in joint training programmes for...,NaN,NaN,NaN,NaN,NaN,NaN,Sharing best practices and teaching methodolog...
2,3,Kasereka Luhindi Jean Louis,Democratic Republic of Congo (DRC),Centre de Formation Professionnelle Don Bosco ...,Local Job Services Officer,No,No,NaN,4-6,Yes,...,NaN,Sharing best practices and teaching methodologies,NaN,Accessing shared resources and materials (curr...,NaN,Contributing to joint advocacy and policy init...,Taking part in regional or national TVET forums,NaN,NaN,Sharing best practices and teaching methodolog...
3,4,RANDZAMAMPIANINA Andrivololomanitra,Madagascar,Centre de Formation au Travail Don Bosco Fiana...,Local Job Services Officer,No,Yes,Administration,4-6,Yes,...,Travail décent pour les jeunes \nOrientation e...,Sharing best practices and teaching methodologies,Participating in joint training programmes for...,Accessing shared resources and materials (curr...,NaN,NaN,Taking part in regional or national TVET forums,NaN,NaN,Sharing best practices and teaching methodolog...
4,5,NaN,South Sudan,Don Bosco VTC Juba,National Job Services Officer,No,No,NaN,1-3,Yes,...,Ministry of labour \nMinistry of higher education,NaN,NaN,NaN,NaN,NaN,NaN,They do not collaborate,NaN,They do not collaborate


In [52]:
# ===========================================================
# 📊 DBTA GP2 Job Services Officers Baseline Dashboard
# ===========================================================

import pandas as pd
import plotly.express as px
from ipywidgets import interact, Dropdown, HBox, HTML
from IPython.display import display
from datetime import date

# ===========================================================
# 🏗️ Don Bosco Tech Africa Branded Header
# ===========================================================

today = date.today().strftime("%B %d, %Y")

header_html = f"""
<div style="
    background-color:#004E8C;
    color:white;
    padding:22px 30px;
    border-radius:10px;
    box-shadow:0 3px 10px rgba(0,0,0,0.15);
    font-family:Calibri, Arial, sans-serif;
    margin-bottom:25px;
">
    <div style="display:flex; align-items:center; justify-content:space-between;">
        <!-- Left Side -->
        <div style="flex:1;">
            <h1 style="margin:0; font-size:28px; font-weight:600;">DBTA GP2 Job Services Officers Survey Dashboard</h1>
            <p style="margin:5px 0 0 0; font-size:15px; color:#f2f2f2;">
                Baseline Analysis | <span style="font-style:italic;">Empowering Youth through Quality TVET</span>
            </p>
        </div>

        <!-- Right Side (Date + Local Logo) -->
        <div style="text-align:right; flex-shrink:0;">
            <p style="margin:0; font-size:14px; color:#ddd;">Generated on {today}</p>
            <div style="margin-top:5px;">
                <img src="DonBoscoTechAfricaLogo.png"
                     alt="DBTA Logo" width="120" style="border-radius:5px;">
            </div>
        </div>
    </div>
</div>
"""

display(HTML(header_html))

# ===========================================================
# 📥 Load and Prepare Data
# ===========================================================

country_col = "Country"

# ✅ Automatically detect Yes/No questions
yes_no_cols = []
for col in data.columns:
    unique_vals = data[col].dropna().astype(str).str.lower().unique()
    if set(unique_vals).issubset({'yes', 'no'}):
        yes_no_cols.append(col)

# Dropdown slicers
country_options = ["All"] + sorted(data[country_col].dropna().unique().tolist())
country_slicer = Dropdown(options=country_options, description="Country:")
question_slicer = Dropdown(options=yes_no_cols, description="Yes/No Question:")

# ===========================================================
# ⚙️ Helper Function for KPI Boxes
# ===========================================================

def create_kpi(label, value, color):
    return HTML(f"""
    <div style='
        display:inline-block;
        width:230px;
        background-color:{color};
        color:white;
        text-align:center;
        padding:10px;
        margin:5px;
        border-radius:10px;
        box-shadow:2px 2px 5px rgba(0,0,0,0.2);
    '>
        <div style='font-size:15px; font-weight:500;'>{label}</div>
        <div style='font-size:22px; font-weight:bold;'>{value}</div>
    </div>
    """)

# ===========================================================
# 🧭 Interactive Dashboard
# ===========================================================

@interact(country=country_slicer, question=question_slicer)
def update_dashboard(country, question):
    # --- Filter by country ---
    if country != "All":
        filtered = data[data[country_col] == country]
    else:
        filtered = data.copy()

    # --- KPI Calculations ---
    total = len(filtered)
    yes_count = (filtered[question].astype(str).str.lower() == "yes").sum()
    no_count = (filtered[question].astype(str).str.lower() == "no").sum()
    yes_pct = round((yes_count / total) * 100, 1) if total > 0 else 0
    no_pct = round((no_count / total) * 100, 1) if total > 0 else 0

    # --- KPI Boxes ---
    kpi_boxes = HBox([
        create_kpi("Selected Country", country if country != "All" else "All Countries", "#004E8C"),
        create_kpi("Total Respondents", f"{total}", "#004E8C"),
        create_kpi("% Satisfied (Yes)", f"{yes_pct}%", "#00A859"),
        create_kpi("% Unsatisfied (No)", f"{no_pct}%", "#E66225")
    ])
    display(kpi_boxes)

    # =====================================================
    # 🎨 DBTA Themed Visualization
    # =====================================================
    dbta_colors = {'Yes': '#004E8C', 'No': '#E66225'}

    fig = px.bar(
        x=["Yes", "No"], 
        y=[yes_count, no_count],
        color=["Yes", "No"],
        color_discrete_map=dbta_colors,
        title=f"<b>{question}</b><br><span style='font-size:14px; color:#555;'>({country if country != 'All' else 'All Countries'})</span>",
        text=[f"{yes_pct}%", f"{no_pct}%"],
        labels={'x': 'Response', 'y': 'Number of Respondents'}
    )

    fig.update_traces(
        textposition="outside",
        textfont_size=14,
        textfont_color="#004E8C",
        marker_line_color='white',
        marker_line_width=1.5,
        width=0.35
    )

    fig.update_layout(
        title_x=0.5,
        title_font=dict(size=17, color='#004E8C', family="Calibri"),
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(family="Calibri", size=13, color='#333'),
        xaxis=dict(showgrid=False, showline=False, tickfont=dict(size=13)),
        yaxis=dict(showgrid=True, gridcolor='#E5ECF6', zeroline=False, title=None),
        margin=dict(l=60, r=60, t=120, b=60),
        uniformtext_minsize=12,
        uniformtext_mode='hide'
    )

    fig.update_yaxes(range=[0, max(yes_count, no_count) * 1.25])
    fig.show()

    # =====================================================
    # 🧾 Professional Summary Card
    # =====================================================
    if yes_pct >= 80:
        rec = (
            "Results indicate strong engagement and alignment with DBTA’s operational goals. "
            "This area shows consistent awareness and implementation across centres. "
            "Focus can now shift toward sustaining quality and sharing best practices."
        )
    elif 50 <= yes_pct < 80:
        rec = (
            "Moderate response rates suggest mixed levels of engagement. "
            "Strengthening communication, ongoing coaching, and access to tools like the JSO Quality Manual "
            "could boost overall satisfaction."
        )
    else:
        rec = (
            "Low agreement highlights a clear area for improvement. "
            "It may require targeted interventions, additional sensitization, or logistical support. "
            "Follow-up surveys and feedback sessions could help uncover specific gaps."
        )

    accent_blue = "#004E8C"
    accent_gray = "#f5f7fa"

    summary_html = f"""
    <div style="
        background-color:{accent_gray};
        border-left:6px solid {accent_blue};
        border-radius:10px;
        padding:18px 24px;
        margin-top:25px;
        box-shadow:1px 2px 6px rgba(0,0,0,0.1);
        font-family:Calibri, Arial, sans-serif;
    ">
        <h3 style="color:{accent_blue}; margin-top:0;">📋 Summary</h3>
        <p style="font-size:15px; color:#333; line-height:1.5;">
            The survey question 
            <span style="font-weight:600;">“{question}”</span> received 
            <span style="font-weight:600;">{total}</span> responses 
            from <span style="font-weight:600;">{country if country != 'All' else 'all participating countries'}</span>.
            Among them, <b>{yes_count}</b> respondents 
            (<b>{yes_pct}%</b>) answered <span style="color:#00A859;">Yes</span>, 
            while <b>{no_count}</b> respondents (<b>{no_pct}%</b>) answered 
            <span style="color:#E66225;">No</span>.
        </p>
        <hr style="border:none; border-top:1px solid #ddd; margin:12px 0;">
        <h3 style="color:{accent_blue}; margin-bottom:6px;">💡 Recommendation</h3>
        <p style="font-size:15px; color:#333; line-height:1.6; margin-bottom:0;">
            {rec}
        </p>
    </div>
    """
    display(HTML(summary_html))

    # =====================================================
    # 🦶 Branded Footer (inside function to load properly)
    # =====================================================
    footer_html = """
    <div style="
        background-color:#004E8C;
        color:white;
        padding:16px 25px;
        border-radius:10px;
        box-shadow:0 -3px 10px rgba(0,0,0,0.15);
        font-family:Calibri, Arial, sans-serif;
        text-align:center;
        margin-top:40px;
    ">
        <p style="margin:4px 0; font-size:14px; letter-spacing:0.3px;">
            © <b>Don Bosco Tech Africa</b> | Data Analytics & Research Unit
        </p>
    </div>
    """
    display(HTML(footer_html))


HTML(value='\n<div style="\n    background-color:#004E8C;\n    color:white;\n    padding:22px 30px;\n    borde…

interactive(children=(Dropdown(description='Country:', options=('All', 'Angola', 'Burkina Faso', 'Burundi', 'C…

In [53]:
# ===========================================================
# 📊 DBTA GP2 JSO Survey — Rating Questions Dashboard (Fixed)
# ===========================================================

import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact, Dropdown, HBox
from IPython.display import display, HTML

# ===========================================================
# 🧭 Strict Filtering for Rating Questions (1–5 scale)
# ===========================================================
exclude_keywords = ["name", "respondent", "email", "phone", "id", "contact"]
filtered_cols = [
    c for c in data.columns
    if not any(k in c.lower() for k in exclude_keywords)
]

rating_cols = []
for col in filtered_cols:
    series = data[col].dropna()
    try:
        numeric_vals = pd.to_numeric(series, errors='coerce').dropna()
        if len(numeric_vals) >= 0.8 * len(series) and numeric_vals.between(1, 5).all():
            rating_cols.append(col)
    except Exception:
        continue

# --- Dropdown slicers ---
country_options = ["All"] + sorted(data["Country"].dropna().unique().tolist())
country_slicer = Dropdown(options=country_options, description="Country:")
rating_slicer = Dropdown(options=rating_cols, description="Rating Question:")

# ===========================================================
# ⚙️ KPI Box Creator (Now Using widgets.HTML)
# ===========================================================
def create_kpi(label, value, color):
    return widgets.HTML(f"""
    <div style='
        display:inline-block;
        width:230px;
        background-color:{color};
        color:white;
        text-align:center;
        padding:10px;
        margin:5px;
        border-radius:10px;
        box-shadow:2px 2px 5px rgba(0,0,0,0.2);
    '>
        <div style='font-size:15px; font-weight:500;'>{label}</div>
        <div style='font-size:22px; font-weight:bold;'>{value}</div>
    </div>
    """)

# ===========================================================
# 📈 Interactive Rating Dashboard
# ===========================================================
@interact(country=country_slicer, question=rating_slicer)
def update_rating_dashboard(country, question):
    # --- Filter data by country ---
    if country != "All":
        filtered = data[data["Country"] == country]
    else:
        filtered = data.copy()

    # --- Clean and convert ratings ---
    filtered = filtered.dropna(subset=[question])
    filtered[question] = pd.to_numeric(filtered[question], errors='coerce')
    filtered = filtered.dropna(subset=[question])

    # --- Aggregation ---
    rating_counts = filtered[question].value_counts().sort_index()
    total = rating_counts.sum()
    avg_rating = round(filtered[question].mean(), 2)
    high_pct = round(((filtered[question] >= 4).sum() / total) * 100, 1) if total > 0 else 0

    # --- KPI Boxes (Now Valid Widgets) ---
    kpi_boxes = HBox([
        create_kpi("Selected Country", country if country != "All" else "All Countries", "#004E8C"),
        create_kpi("Total Respondents", f"{total}", "#004E8C"),
        create_kpi("Average Rating", f"{avg_rating}", "#00A859"),
        create_kpi("% Rated 4 or 5", f"{high_pct}%", "#E66225")
    ])
    display(kpi_boxes)

    # =====================================================
    # 🎨 Don Bosco–Styled Donut Chart
    # =====================================================
    dbta_colors = ["#004E8C", "#005FA3", "#0074C1", "#00A859", "#E66225"]

    fig = px.pie(
        names=rating_counts.index.astype(str),
        values=rating_counts.values,
        hole=0.45,
        color=rating_counts.index.astype(str),
        color_discrete_sequence=dbta_colors,
        title=f"<b>{question}</b><br><span style='font-size:14px; color:#555;'>({country if country != 'All' else 'All Countries'})</span>"
    )

    fig.update_traces(
        textinfo='percent+label',
        textfont_size=14,
        hoverinfo='label+percent',
        marker=dict(line=dict(color='white', width=2))
    )

    fig.update_layout(
        title_x=0.5,
        title_font=dict(size=17, color='#004E8C', family="Calibri"),
        paper_bgcolor='white',
        plot_bgcolor='white',
        font=dict(family="Calibri", size=13, color='#333'),
        margin=dict(l=50, r=50, t=100, b=50)
    )
    fig.show()

    # =====================================================
    # 🧾 Professional Summary Card
    # =====================================================
    if avg_rating >= 4.0:
        rec = (
            "High satisfaction levels indicate strong alignment with DBTA quality standards. "
            "Continue reinforcing these best practices across other centres to sustain excellence."
        )
    elif 3.0 <= avg_rating < 4.0:
        rec = (
            "Moderate satisfaction suggests there is room for improvement. "
            "Targeted feedback sessions and enhanced implementation of the JSO Manual could elevate performance."
        )
    else:
        rec = (
            "Low satisfaction ratings highlight potential training or communication gaps. "
            "DBTA should prioritize capacity building and closer monitoring in these areas."
        )

    summary_html = f"""
    <div style="
        background-color:#f5f7fa;
        border-left:6px solid #004E8C;
        border-radius:10px;
        padding:18px 24px;
        margin-top:25px;
        box-shadow:1px 2px 6px rgba(0,0,0,0.1);
        font-family:Calibri, Arial, sans-serif;
    ">
        <h3 style="color:#004E8C; margin-top:0;">📋 Summary</h3>
        <p style="font-size:15px; color:#333; line-height:1.5;">
            The survey question 
            <span style="font-weight:600;">“{question}”</span> received 
            <span style="font-weight:600;">{total}</span> valid responses 
            from <span style="font-weight:600;">{country if country != 'All' else 'all participating countries'}</span>.
            The average rating is <b>{avg_rating}</b>, and <b>{high_pct}%</b> of respondents rated 4 or 5, 
            indicating <span style="color:#00A859;">{ 'high satisfaction' if avg_rating >= 4 else 'moderate satisfaction' if avg_rating >= 3 else 'low satisfaction' }</span>.
        </p>
        <hr style="border:none; border-top:1px solid #ddd; margin:12px 0;">
        <h3 style="color:#004E8C; margin-bottom:6px;">💡 Recommendation</h3>
        <p style="font-size:15px; color:#333; line-height:1.6; margin-bottom:0;">
            {rec}
        </p>
    </div>
    """
    display(HTML(summary_html))


interactive(children=(Dropdown(description='Country:', options=('All', 'Angola', 'Burkina Faso', 'Burundi', 'C…

In [54]:
# ===========================================================
# 📊 DBTA GP2 Job Services Officers Unified Survey Dashboard
# ===========================================================

import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact, Dropdown, HBox
from IPython.display import display, HTML
from datetime import date

# ===========================================================
# 🧱 Don Bosco Header
# ===========================================================
today = date.today().strftime("%B %d, %Y")

header_html = f"""
<div style="
    background-color:#004E8C;
    color:white;
    padding:22px 30px;
    border-radius:10px;
    box-shadow:0 3px 10px rgba(0,0,0,0.15);
    font-family:Calibri, Arial, sans-serif;
    margin-bottom:25px;
">
    <div style="display:flex; align-items:center; justify-content:space-between;">
        <div style="flex:1;">
            <h1 style="margin:0; font-size:28px; font-weight:600;">DBTA GP2 Job Services Officers Survey Dashboard</h1>
            <p style="margin:5px 0 0 0; font-size:15px; color:#f2f2f2;">
                Baseline Analysis | <span style="font-style:italic;">Empowering Youth through Quality TVET</span>
            </p>
        </div>
        <div style="text-align:right; flex-shrink:0;">
            <p style="margin:0; font-size:14px; color:#ddd;">Generated on {today}</p>
            <div style="margin-top:5px;">
                <img src="DonBoscoTechAfricaLogo.png" alt="DBTA Logo" width="120" style="border-radius:5px;">
            </div>
        </div>
    </div>
</div>
"""
display(HTML(header_html))

# ===========================================================
# 📥 Identify Question Types (Yes/No & Rating)
# ===========================================================
country_col = "Country"
exclude_keywords = ["name", "respondent", "email", "phone", "id", "contact"]
filtered_cols = [c for c in data.columns if not any(k in c.lower() for k in exclude_keywords)]

# --- Detect Yes/No questions ---
yes_no_cols = []
for col in filtered_cols:
    unique_vals = data[col].dropna().astype(str).str.lower().unique()
    if set(unique_vals).issubset({'yes', 'no'}):
        yes_no_cols.append(col)

# --- Detect Rating (1–5) questions ---
rating_cols = []
for col in filtered_cols:
    series = data[col].dropna()
    try:
        numeric_vals = pd.to_numeric(series, errors='coerce').dropna()
        if len(numeric_vals) >= 0.7 * len(series) and numeric_vals.between(1, 5).all():
            rating_cols.append(col)
    except Exception:
        continue

# ===========================================================
# ⚙️ Helper — DBTA KPI Box
# ===========================================================
def create_kpi(label, value, color):
    return widgets.HTML(f"""
    <div style='
        display:inline-block;
        width:230px;
        background-color:{color};
        color:white;
        text-align:center;
        padding:10px;
        margin:5px;
        border-radius:10px;
        box-shadow:2px 2px 5px rgba(0,0,0,0.2);
    '>
        <div style='font-size:15px; font-weight:500;'>{label}</div>
        <div style='font-size:22px; font-weight:bold;'>{value}</div>
    </div>
    """)

# ===========================================================
# 🧩 Dropdowns (Type + Country + Question)
# ===========================================================
question_type_dropdown = Dropdown(
    options=["Yes/No", "Rating (1–5)"],
    description="Question Type:"
)
country_dropdown = Dropdown(
    options=["All"] + sorted(data[country_col].dropna().unique().tolist()),
    description="Country:"
)
question_dropdown = Dropdown(options=yes_no_cols, description="Question:")

# ===========================================================
# 🧭 Unified Dashboard Logic
# ===========================================================
def update_dashboard(question_type, country, question):
    display(HTML(f"<h3 style='color:#004E8C; font-family:Calibri;'>📊 {question_type} Analysis</h3>"))

    if question_type == "Yes/No":
        filtered = data.copy() if country == "All" else data[data[country_col] == country]
        total = len(filtered)
        yes_count = (filtered[question].astype(str).str.lower() == "yes").sum()
        no_count = (filtered[question].astype(str).str.lower() == "no").sum()
        yes_pct = round((yes_count / total) * 100, 1) if total > 0 else 0
        no_pct = round((no_count / total) * 100, 1) if total > 0 else 0

        # KPI
        kpi = HBox([
            create_kpi("Country", country if country != "All" else "All", "#004E8C"),
            create_kpi("Total Responses", total, "#004E8C"),
            create_kpi("% Yes", f"{yes_pct}%", "#00A859"),
            create_kpi("% No", f"{no_pct}%", "#E66225")
        ])
        display(kpi)

        # Visualization
        fig = px.bar(
            x=["Yes", "No"], y=[yes_count, no_count],
            color=["Yes", "No"],
            color_discrete_map={'Yes': '#004E8C', 'No': '#E66225'},
            text=[f"{yes_pct}%", f"{no_pct}%"],
            title=f"<b>{question}</b><br><span style='font-size:14px; color:#555;'>({country})</span>"
        )
        fig.update_traces(textposition="outside", width=0.35)
        fig.update_layout(title_x=0.5, plot_bgcolor='white', paper_bgcolor='white')
        fig.show()

        # Summary + Recommendation
        if yes_pct >= 80:
            rec = "Strong alignment with DBTA’s objectives. This area shows excellence and can be a model for replication."
        elif 50 <= yes_pct < 80:
            rec = "Moderate engagement. Reinforcing capacity-building, mentoring, and improved access to tools may enhance outcomes."
        else:
            rec = "Low engagement detected. This needs targeted sensitization and support to improve awareness and participation."

        summary_html = f"""
        <div style="
            background-color:#f5f7fa;
            border-left:6px solid #004E8C;
            border-radius:10px;
            padding:18px 24px;
            margin-top:25px;
            box-shadow:1px 2px 6px rgba(0,0,0,0.1);
            font-family:Calibri, Arial, sans-serif;
        ">
            <h3 style="color:#004E8C; margin-top:0;">📋 Summary</h3>
            <p style="font-size:15px; color:#333;">
                The question <b>“{question}”</b> received <b>{total}</b> responses 
                from <b>{country if country != 'All' else 'all countries'}</b>. 
                Out of these, <b>{yes_count}</b> respondents ({yes_pct}%) answered <span style="color:#00A859;">Yes</span> 
                and <b>{no_count}</b> respondents ({no_pct}%) answered <span style="color:#E66225;">No</span>.
            </p>
            <hr style="border:none; border-top:1px solid #ddd; margin:12px 0;">
            <h3 style="color:#004E8C; margin-bottom:6px;">💡 Recommendation</h3>
            <p style="font-size:15px; color:#333;">{rec}</p>
        </div>
        """
        display(HTML(summary_html))

    else:  # Rating (1–5)
        filtered = data.copy() if country == "All" else data[data[country_col] == country]
        filtered = filtered.dropna(subset=[question])
        filtered[question] = pd.to_numeric(filtered[question], errors='coerce')
        filtered = filtered.dropna(subset=[question])

        rating_counts = filtered[question].value_counts().sort_index()
        total = rating_counts.sum()
        avg_rating = round(filtered[question].mean(), 2)
        high_pct = round(((filtered[question] >= 4).sum() / total) * 100, 1) if total > 0 else 0

        kpi = HBox([
            create_kpi("Country", country if country != "All" else "All", "#004E8C"),
            create_kpi("Total Responses", total, "#004E8C"),
            create_kpi("Average Rating", f"{avg_rating}", "#00A859"),
            create_kpi("% Rated 4 or 5", f"{high_pct}%", "#E66225")
        ])
        display(kpi)

        fig = px.pie(
            names=rating_counts.index.astype(str),
            values=rating_counts.values,
            hole=0.45,
            color_discrete_sequence=["#004E8C", "#005FA3", "#0074C1", "#00A859", "#E66225"],
            title=f"<b>{question}</b><br><span style='font-size:14px; color:#555;'>({country})</span>"
        )
        fig.update_traces(textinfo='percent+label', textfont_size=14)
        fig.update_layout(title_x=0.5, plot_bgcolor='white', paper_bgcolor='white')
        fig.show()

        # Summary + Recommendation
        if avg_rating >= 4:
            rec = "High satisfaction levels reflect strong operational standards. Sustain mentorship and peer-learning to maintain excellence."
        elif 3 <= avg_rating < 4:
            rec = "Moderate satisfaction suggests room for growth. Encourage feedback and align training materials with field realities."
        else:
            rec = "Low satisfaction indicates improvement areas in content delivery, accessibility, or training relevance."

        summary_html = f"""
        <div style="
            background-color:#f5f7fa;
            border-left:6px solid #004E8C;
            border-radius:10px;
            padding:18px 24px;
            margin-top:25px;
            box-shadow:1px 2px 6px rgba(0,0,0,0.1);
            font-family:Calibri, Arial, sans-serif;
        ">
            <h3 style="color:#004E8C; margin-top:0;">📋 Summary</h3>
            <p style="font-size:15px; color:#333;">
                The question <b>“{question}”</b> had <b>{total}</b> valid responses 
                from <b>{country if country != 'All' else 'all countries'}</b>. 
                The average rating is <b>{avg_rating}</b>, and <b>{high_pct}%</b> of respondents rated 4 or 5.
            </p>
            <hr style="border:none; border-top:1px solid #ddd; margin:12px 0;">
            <h3 style="color:#004E8C; margin-bottom:6px;">💡 Recommendation</h3>
            <p style="font-size:15px; color:#333;">{rec}</p>
        </div>
        """
        display(HTML(summary_html))

# ===========================================================
# 🔁 Auto-update question list
# ===========================================================
def update_question_list(change):
    question_dropdown.options = yes_no_cols if question_type_dropdown.value == "Yes/No" else rating_cols

question_type_dropdown.observe(update_question_list, names='value')

# ===========================================================
# 🚀 Run Interactive Dashboard
# ===========================================================
interact(update_dashboard,
         question_type=question_type_dropdown,
         country=country_dropdown,
         question=question_dropdown)


interactive(children=(Dropdown(description='Question Type:', options=('Yes/No', 'Rating (1–5)'), value='Yes/No…

<function __main__.update_dashboard(question_type, country, question)>

In [55]:
# ===========================================================
# 🌍 DBTA GP2 Collaboration Dashboard (Smart Detection Fixed)
# ===========================================================

import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# --- Identify columns ---
collab_cols = [col for col in data.columns if "collaborate" in col.lower() and "[" in col]
collab_labels = [col.split("[")[-1].replace("]", "").strip() for col in collab_cols]
collab_map = dict(zip(collab_labels, collab_cols))
country_col = [c for c in data.columns if "country" in c.lower()][0]

# --- Slicers ---
country_selector = widgets.Dropdown(
    options=["All"] + sorted(data[country_col].dropna().unique().tolist()),
    description="🌍 Country:",
    layout=widgets.Layout(width="40%"),
    style={'description_width': 'initial'}
)
collab_selector = widgets.Dropdown(
    options=collab_labels,
    description="🤝 Collaboration Type:",
    layout=widgets.Layout(width="60%"),
    style={'description_width': 'initial'}
)

# ===========================================================
# ⚙️ Dashboard Logic
# ===========================================================
def update_collaboration_dashboard(selected_country, selected_collab):
    clear_output(wait=True)
    display(HTML("<h2 style='color:#004E8C;font-family:Calibri;margin-bottom:10px;'>🌍 Don Bosco TVET Collaboration Dashboard</h2>"))
    display(widgets.HBox([country_selector, collab_selector]))

    col = collab_map[selected_collab]
    df = data.copy()

    # --- Normalize responses ---
    df[col] = df[col].astype(str).str.strip().str.lower()

    # ✅ Smart detection logic
    positive_values = ['yes', 'y', 'true', '1', 'x', 'selected', 'checked', '✓', '√', selected_collab.lower()]
    df_filtered = df[df[col].isin(positive_values)]

    # --- Filter by country if needed ---
    if selected_country != "All":
        df_filtered = df_filtered[df_filtered[country_col] == selected_country]

    # --- Check if any results exist ---
    if df_filtered.empty:
        display(HTML(f"""
        <div style='background-color:#fff3cd; border-left:6px solid #ffcc00;
                    padding:15px; border-radius:8px; font-family:Calibri; margin-top:20px;'>
            ⚠️ No respondents in <b>{selected_country if selected_country != "All" else "the dataset"}</b> selected
            “{selected_collab}”.
        </div>"""))
        return

    # =======================================================
    # 📊 Aggregation by Country
    # =======================================================
    grouped = df_filtered.groupby(country_col).size().reset_index(name="Count")

    # =======================================================
    # 🎨 Don Bosco–Styled Bar Chart
    # =======================================================
    fig = px.bar(
        grouped,
        x=country_col,
        y="Count",
        text="Count",
        color=country_col,
        title=f"<b>Respondents Selecting “{selected_collab}”</b><br><span style='font-size:14px; color:#555;'>({selected_country if selected_country != 'All' else 'All Countries'})</span>",
        color_discrete_sequence=px.colors.qualitative.Vivid
    )

    fig.update_traces(
        textposition="outside",
        marker_line_color='white',
        marker_line_width=1.3
    )
    fig.update_layout(
        title_x=0.5,
        title_font=dict(size=18, color='#004E8C', family="Calibri"),
        xaxis=dict(title=None, tickangle=25, tickfont=dict(size=12)),
        yaxis=dict(title="Number of Respondents"),
        plot_bgcolor='white',
        paper_bgcolor='white',
        showlegend=False,
        margin=dict(t=70, b=50)
    )
    fig.show()

    # =======================================================
    # 📋 Summary + Recommendation
    # =======================================================
    total_resp = grouped["Count"].sum()
    top_country = grouped.sort_values("Count", ascending=False).iloc[0][country_col]
    top_count = grouped.sort_values("Count", ascending=False).iloc[0]["Count"]

    if total_resp >= 15:
        reco = f"Strong collaboration patterns exist across regions. {top_country} leads with best practice in <b>{selected_collab.lower()}</b>. DBTA should document and replicate their model regionally."
    elif 5 <= total_resp < 15:
        reco = f"Moderate adoption of <b>{selected_collab.lower()}</b> across countries. Focus should be on promoting structured peer exchanges and policy-level incentives."
    else:
        reco = f"Limited engagement reported in <b>{selected_collab.lower()}</b>. Recommend targeted sensitization sessions and follow-up assessments to improve collaboration culture."

    summary_html = f"""
    <div style="
        background-color:#f5f7fa;
        border-left:6px solid #004E8C;
        border-radius:10px;
        padding:18px 24px;
        margin-top:25px;
        box-shadow:1px 2px 6px rgba(0,0,0,0.1);
        font-family:Calibri, Arial, sans-serif;
    ">
        <h3 style="color:#004E8C; margin-top:0;">📋 Summary</h3>
        <p style="font-size:15px; color:#333;">
            The collaboration area <b>“{selected_collab}”</b> was reported by 
            <b>{total_resp}</b> respondents across 
            <b>{'all countries' if selected_country == 'All' else selected_country}</b>.
            The most active country is <b>{top_country}</b> with <b>{top_count}</b> mentions.
        </p>
        <hr style="border:none; border-top:1px solid #ddd; margin:12px 0;">
        <h3 style="color:#004E8C; margin-bottom:6px;">💡 Recommendation</h3>
        <p style="font-size:15px; color:#333; line-height:1.6; margin-bottom:0;">
            {reco}
        </p>
    </div>
    """
    display(HTML(summary_html))

# ===========================================================
# 🚀 Launch Dashboard
# ===========================================================
widgets.interact(update_collaboration_dashboard, selected_country=country_selector, selected_collab=collab_selector);


interactive(children=(Dropdown(description='🌍 Country:', layout=Layout(width='40%'), options=('All', 'Angola',…